# SFT of SmilLM2

## Libraries

In [1]:
import os
import torch
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer
import re
import random
from rich.console import Console
from rich.panel import Panel
from rich.rule import Rule

/Data/AxelDlv/condaenvs/SJTU/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Global variables

In [3]:
model_id = "HuggingFaceTB/SmolLM2-360M-Instruct"
dataset_id = "AxelDlv00/EULAI"
output_dir = "./EULAI-SmolLM2-Instruct"

## Dataset preparation

In [4]:
dataset = load_dataset(dataset_id)

In [5]:
def preprocess_function(example):
    return {
        "messages": [
            {"role": "user", "content": example["policy"]},
            {"role": "assistant", "content": example["summary"]}
        ]
    }

In [6]:
dataset = dataset.map(preprocess_function, remove_columns=dataset["train"].column_names)

dataset["train"] = dataset["train"].shuffle(seed=42).select(range(500))
dataset["test"] = dataset["test"].shuffle(seed=42).select(range(min(50, len(dataset["test"]))))

# Tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

## Training preparation

In [8]:
training_args = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="steps",
    eval_steps=50,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1, 
    max_length=2048,            
    packing=False,              
    
    dataset_kwargs = {
        "add_special_tokens": False,  
    },
    
    bf16=torch.cuda.is_bf16_supported() if torch.cuda.is_available() else False,
    fp16=not (torch.cuda.is_bf16_supported() if torch.cuda.is_available() else False),
)

In [9]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [10]:
trainer = SFTTrainer(
    model=model_id,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    args=training_args,
    processing_class=tokenizer, 
)

/Data/AxelDlv/condaenvs/SJTU/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/Data/AxelDlv/condaenvs/SJTU/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
Truncating train dataset: 100%|██████████| 500/500 [00:00<00:00, 63463.52 examples/s]
The model is already on multiple devices. Skipping the move to device specified in `args`.


## Training

In [11]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=48, training_loss=2.013322949409485, metrics={'train_runtime': 726.058, 'train_samples_per_second': 2.066, 'train_steps_per_second': 0.066, 'total_flos': 5959410647040000.0, 'train_loss': 2.013322949409485})

## Print results

In [12]:
def parse_generative_output(text: str):
    """Extrait les points clés du texte généré par le modèle."""
    results = []
    pattern = re.compile(r"^-\s*\[(BAD|GOOD|NEUTRAL|BLOCKER)\]\s*:\s*([^:]+):\s*(.+)$", re.MULTILINE)
    
    for match in pattern.finditer(text):
        results.append({
            "label": match.group(1).upper(),
            "title": match.group(2).strip(),
            "explanation": match.group(3).strip()
        })
    return results

def visualize_inference(trainer, tokenizer, dataset_id, num_samples=3):
    console = Console()
    console.print(Rule("[bold magenta]Visualisation Inférence SmolLM2[/bold magenta]"))
    try:
        raw_test_dataset = load_dataset(dataset_id, split="test")
    except:
        raw_test_dataset = load_dataset(dataset_id)["test"]
        
    indices = random.sample(range(len(raw_test_dataset)), min(num_samples, len(raw_test_dataset)))
    selected_items = [raw_test_dataset[i] for i in indices]
    
    model = trainer.model
    model.eval()
    
    for item in selected_items:
        policy_text = item['policy']
        
        messages = [{"role": "user", "content": policy_text}]
        
        inputs = tokenizer.apply_chat_template(
            messages, 
            tokenize=True, 
            add_generation_prompt=True, 
            return_tensors="pt"
        ).to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(
                inputs, 
                max_new_tokens=512, 
                temperature=0.1, 
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        # Décodage (on saute les tokens spéciaux)
        full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extraction de la partie "assistant" uniquement (approximatif via parsing)
        # On parse directement tout le texte généré pour trouver nos patterns
        points = parse_generative_output(full_output)
        
        # 4. Affichage Rich
        console.print(f"\n[bold underline white on blue] TEST SAMPLE (ID: {item.get('id', 'N/A')[:6]}...) [/bold underline white on blue]")
        
        if not points:
             # Si le modèle a échoué à suivre le format, on affiche le brut
             console.print(Panel(full_output, title="⚠️ Raw Output (Format non détecté)", border_style="red"))
        
        for p in points:
            label = p['label']
            title = p['title']
            expl = p['explanation']
            
            # Mapping des couleurs comme dans votre script original
            if label == "BLOCKER":
                color = "white on red" 
                icon = "⛔"
                border = "red"
            elif label == "BAD":
                color = "red"
                icon = "❌"
                border = "red"
            elif label == "GOOD":
                color = "green"
                icon = "✅"
                border = "green"
            else:
                color = "yellow"
                icon = "ℹ️"
                border = "yellow"
            
            p_content = f"[{color}][bold]{title}[/bold][/{color}]\n[white]{expl}[/white]"
            
            console.print(Panel(
                p_content,
                title=f"{icon} {label}",
                title_align="left",
                border_style=border,
                width=100
            ))

In [13]:
visualize_inference(trainer, tokenizer, dataset_id)

───────────────────────────────────────── Visualisation Inférence SmolLM2 ─────────────────────────────────────────

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 TEST SAMPLE (ID: 7589fc...) 

╭─────────────────────────────────────── ⚠️ Raw Output (Format non détecté) ───────────────────────────────────────╮
│ system                                                                                                          │
│ You are a helpful AI assistant named SmolLM, trained by Hugging Face                                            │
│ user                                                                                                            │
│ We respect your privacy                                                                                         │
│                                                                                                                 │
│ We use cookies to personalise your experience, marketing, and collect analytics.                                │
│                                                                                                                 │
│ [Learn more](https://www.maplin.co.uk/policies/privacy-policy)  [Google's Privacy                               │
│ Terms](https://business.safety.google/privacy/)                                                                 │
│  [Skip to content](https://www.maplin.co.uk/policies/terms-of-service#MainContent)                              │
│ [For business](https://www.maplin.co.uk/pages/business)                                                         │
│                                                                                                                 │
│ 📣 Sale season may be over, but we still have plenty of  [Offers](https://www.maplin.co.uk/collections/offers)  │
│ 📣                                                                                                              │
│                                                                                                                 │
│ [Help](https://www.maplin.co.uk/pages/help)                                                                     │
│                                                                                                                 │
│ # Terms of service                                                                                              │
│                                                                                                                 │
│ **Who are we? 💙**                                                                                              │
│                                                                                                                 │
│ We're Digital-First Retail Limited (company number 11441187), a registered company in England and Wales, based  │
│ at First Floor, Unit 5, Century Point, Halifax Road, High Wycombe, Bucks, HP12 3SL. Our UK VAT number is GB     │
│ 303179233. When you browse our website and shop online these terms apply. You should read this document along   │
│ with our  [Privacy Policy](https://www.maplin.co.uk/policies/privacy-policy)                                    │
│                                                                                                                 │
│ These terms apply when you shop with us (or access our services and websites (our “Websites”)). You can also    │
│ find out more on our  [Help Centre](https://help.maplin.co.uk/) (which also form part of these terms). We keep  │
│ these terms and Help Centre updated and we amend them every so often.It is important to check back in before    │
│ you shop, as the latest set will apply.                                                                         │
│                                                                                                                 │
│ **You ✌**                                                                                                       │
│                                                                                                                 │
│ To legally place an order on Maplin you need to:        

 TEST SAMPLE (ID: 09284b...) 

╭─────────────────────────────────────── ⚠️ Raw Output (Format non détecté) ───────────────────────────────────────╮
│ system                                                                                                          │
│ You are a helpful AI assistant named SmolLM, trained by Hugging Face                                            │
│ user                                                                                                            │
│ ## Privacy Information                                                                                          │
│                                                                                                                 │
│ Open Social Security does not save the information that you enter into the calculator.                          │
│                                                                                                                 │
│ This website does, however, use Google Analytics, a service which transmits website traffic data to Google      │
│ servers. Google Analytics does not identify individual users or associate your IP address with any other data   │
│ held by Google. This website uses reports provided by Google Analytics to help understand the way in which      │
│ users are using (and arriving at) the website. Google Analytics involves the use of an assortment of cookies.   │
│ For more information about Google Analytics, see here:                                                          │
│                                                                                                                 │
│ (https://www.google.com/policies/privacy/partners/)                                                             │
│                                                                                                                 │
│ If you are interested in opting out of Google Analytics, Google offers                                          │
│ (https://tools.google.com/dlpage/gaoptout)  that allows you to do so.                                           │
│                                                                                                                 │
│ ## Disclaimer                                                                                                   │
│                                                                                                                 │
│ No information provided on this site shall be construed as legal, accounting, investment, financial, tax, or    │
│ other professional advice on any matter. You alone bear the sole responsibility of assessing the merits and     │
│ risks associated with any financial decisions you make.                                                         │
│                                                                                                                 │
│ By using this site, you explicitly agree not to hold Michael Piper, Michael Piper, LLC, or Simple Subjects,     │
│ LLC, liable in any way for any damages arising from decisions you make based on the information made available  │
│ on this site.                                                                                                   │
│                                                                                                                 │
│ Your use of this website and its information does not create any professional services relationship between you │
│ and Michael Piper, between you and Michael Piper, LLC, or between you and Simple Subjects, LLC. Sending an      │
│ email to Michael Piper, Michael Piper, LLC or Simple Subjects, LLC does not create any client relationship.     │
│                                                                                                                 │
│ By providing access to other websites, neither Simple Subjects, LLC nor any of its members makes any            │
│ endorsement whatsoever regarding those sites.        

 TEST SAMPLE (ID: 3f3df8...) 

╭─────────────────────────────────────── ⚠️ Raw Output (Format non détecté) ───────────────────────────────────────╮
│ system                                                                                                          │
│ You are a helpful AI assistant named SmolLM, trained by Hugging Face                                            │
│ user                                                                                                            │
│ # PillPack Terms of Use                                                                                         │
│                                                                                                                 │
│ Last updated:  **April 9, 2020**                                                                                │
│                                                                                                                 │
│ Welcome to PillPack.com. PillPack LLC and/or its affiliates (“PillPack”) provide website features and other     │
│ products and services to you when you visit pillpack.com, contact PillPack, or otherwise interact with          │
│ PillPack, use PillPack products or services, use PillPack applications for mobile, or use any software provided │
│ by PillPack in connection with any of the foregoing (collectively, “PillPack Services”). PillPack provides the  │
│ PillPack Services subject to these Terms of Use.                                                                │
│                                                                                                                 │
│ By using PillPack Services, you agree to these Terms of Use. Please read them carefully. If you do not agree,   │
│ do not use the PillPack Services.                                                                               │
│                                                                                                                 │
│ PillPack may amend any part of these Terms of Use at our sole discretion by posting the revised Terms of Use on │
│ the PillPack website. You should periodically check these Terms of Use for changes. Your continued use of the   │
│ PillPack Services following the posting of any changes to these Terms of Use constitutes your acceptance of the │
│ terms.                                                                                                          │
│                                                                                                                 │
│ Privacy                                                                                                         │
│                                                                                                                 │
│ Our  [Notice of Privacy Practices](https://www.pillpack.com/privacy-notice)  describes how your health          │
│ information may be used and disclosed and how you can get access to this information. The PillPack Services are │
│ also governed by the  [Amazon.com Privacy                                                                       │
│ Notice](https://www.amazon.com/gp/help/customer/display.html?nodeId=GX7NJQ4ZB8MHFRNJ&ref_=footer_privacy)       │
│                                                                                                                 │
│ Content                                                                                                         │
│                                                                                                                 │
│ All information, data, software, photographs, graphics, videos, text, images, typefaces, sounds, logos, and     │
│ other material, including but not limited to the selection, coordination, arrangement, and enhancement of such  │
│ content, contained on any PillPack Service is owned, controlled, or licensed by or to PillPack, and is          │
│ protected by trade dress, copyright, patent, trademar